In [45]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [56]:
#1페이지 크롤링 하기
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 가상 브라우저 헤더 설정
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}

# 빈 리스트 초기화
all_player_data = []

# 1페이지 크롤링
url_1 = 'https://www.koreabaseball.com/Record/Player/HitterBasic/BasicOld.aspx?sort=HRA_RT'
r_1 = requests.get(url_1, headers=headers)
soup_1 = BeautifulSoup(r_1.content, 'html.parser')

In [57]:
# 테이블에서 모든 선수 데이터 추출
player_html_1 = soup_1.select('div.record_result > table > tbody > tr')
for i in range(len(player_html_1)):
    player_td = player_html_1[i].select('td')
    player_line = [td.text.strip() for td in player_td]
    all_player_data.append(player_line)

# DataFrame 생성
player = pd.DataFrame(all_player_data)

In [58]:
# 컬럼명 설정 (1페이지에서 thead 추출)
r = requests.get('https://www.koreabaseball.com/Record/Player/HitterBasic/BasicOld.aspx?sort=HRA_RT', headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')
player_thead = soup.select('div.record_result > table > thead > tr > th')
player_columns = [th.text.strip() for th in player_thead]
player.columns = player_columns

# '순위' 컬럼을 인덱스로 설정
player = player.set_index('순위')

# 결과 확인
print(player)

# CSV 파일로 저장
player.to_csv('kbo_player_data1.csv', encoding='utf-8-sig')

      선수명   팀명    AVG    G   PA   AB    H  2B  3B  HR  RBI  SB  CS  BB HBP  \
순위                                                                           
1    에레디아  SSG  0.360  136  591  541  195  31   1  21  118   4   3  28  13   
2    레이예스   롯데  0.352  144  632  574  202  40   3  15  111   5   0  46   1   
3     김도영  KIA  0.347  141  625  544  189  29  10  38  109  40   4  66   7   
4     구자욱   삼성  0.343  129  568  493  169  39   1  33  115  13   4  55  12   
5     송성문   키움  0.340  142  602  527  179  29   4  19  104  21   0  64   3   
6     홍창기   LG  0.336  139  637  524  176  18   3   5   73  10  11  96  12   
7     로하스   KT  0.329  144  670  572  188  39   1  32  112   2   1  88   6   
8     김선빈  KIA  0.329  116  466  423  139  23   0   9   57   5   1  30   5   
9     박민우   NC  0.328  121  528  457  150  26   2   8   50  32   7  54   9   
10    김혜성   키움  0.326  127  567  509  166  26   4  11   75  30   6  47   4   
11    오스틴   LG  0.319  140  604  527  168  32   3  32  132  12  

In [65]:
#2페이지를 모두 추출하고 싶어요
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# WebDriver 설정 (ChromeDriver 경로 입력)
driver_path = "C://Users//user//jupyter//kbl 데이터 크롤링//chromedriver-win64//chromedriver-win64//chromedriver.exe"
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# KBO 기록 페이지 접속
url = 'https://www.koreabaseball.com/Record/Player/HitterBasic/Detail1.aspx'
driver.get(url)

# 데이터를 저장할 리스트 초기화
all_player_data = []

In [66]:
# 1페이지 데이터 가져오기
time.sleep(2)  # 페이지 로드 대기
soup_1 = BeautifulSoup(driver.page_source, 'html.parser')
player_html_1 = soup_1.select('div.record_result > table > tbody > tr')

for i in range(len(player_html_1)):
    player_td = player_html_1[i].select('td')
    player_line = [td.text.strip() for td in player_td]
    all_player_data.append(player_line)

# 2페이지 버튼 클릭
page_2_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, 'cphContents_cphContents_cphContents_ucPager_btnNo2'))
)
page_2_button.click()

# 2페이지 데이터 가져오기
time.sleep(2)  # 페이지 로드 대기
soup_2 = BeautifulSoup(driver.page_source, 'html.parser')
player_html_2 = soup_2.select('div.record_result > table > tbody > tr')

for i in range(len(player_html_2)):
    player_td = player_html_2[i].select('td')
    player_line = [td.text.strip() for td in player_td]
    all_player_data.append(player_line)

# WebDriver 종료
driver.quit()

# 데이터프레임 생성
player = pd.DataFrame(all_player_data)

# 컬럼명 설정 (1페이지에서 thead 추출)
service = Service(driver_path)
driver = webdriver.Chrome(service=service)
driver.get(url)
time.sleep(2)  # 페이지 로드 대기
soup = BeautifulSoup(driver.page_source, 'html.parser')
player_thead = soup.select('div.record_result > table > thead > tr > th')
player_columns = [th.text.strip() for th in player_thead]
driver.quit()

player.columns = player_columns

# '순위' 컬럼을 인덱스로 설정
player = player.set_index('순위')

# 결과 확인
print(player)

# CSV 파일로 저장
player.to_csv('kbo_player_data.csv', encoding='utf-8-sig')

      선수명   팀명    AVG XBH   GO   AO GO/AO GW RBI  BB/K  P/PA   ISOP     XR  \
순위                                                                           
1    에레디아  SSG  0.360  53  128  154  0.83     13  0.38  3.46  0.177  105.2   
2    레이예스   롯데  0.352  58  168  133  1.26     13  0.56  3.87  0.159  106.2   
3     김도영  KIA  0.347  77  101  151  0.67     15  0.60  3.81  0.300  138.2   
4     구자욱   삼성  0.343  73  128  129  0.99     12  0.75  3.80  0.284  118.5   
5     송성문   키움  0.340  52  136  138  0.99     11  0.78  3.88  0.178  111.4   
6     홍창기   LG  0.336  26  159  100  1.59      6  1.03  4.14  0.074   99.5   
7     로하스   KT  0.329  72  112  161  0.70     12  0.77  4.00  0.240  126.7   
8     김선빈  KIA  0.329  32  139  106  1.31      4  0.77  3.80  0.118   65.1   
9     박민우   NC  0.328  36  106  126  0.84      3  0.68  3.90  0.118   85.4   
10    김혜성   키움  0.326  41  132  156  0.85      9  0.76  3.82  0.132   90.9   
11    오스틴   LG  0.319  67  100  190  0.53     14  0.74  3.78  0.

In [69]:
#투수 포지션도 추출하고 싶어요
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# WebDriver 설정 (ChromeDriver 경로 입력)
driver_path = "C://Users//user//jupyter//kbl 데이터 크롤링//chromedriver-win64//chromedriver-win64//chromedriver.exe"
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# KBO 기록 페이지 접속
url = 'https://www.koreabaseball.com/Record/Player/PitcherBasic/Detail1.aspx'
driver.get(url)

# 데이터를 저장할 리스트 초기화
all_player_data = []

In [70]:
# 1페이지 데이터 크롤링
time.sleep(2)  # 페이지 로드 대기
soup_1 = BeautifulSoup(driver.page_source, 'html.parser')
player_html_1 = soup_1.select('div.record_result > table > tbody > tr')

for i in range(len(player_html_1)):
    player_td = player_html_1[i].select('td')
    player_line = [td.text.strip() for td in player_td]
    all_player_data.append(player_line)

# WebDriver 종료
driver.quit()

# 데이터프레임 생성
player = pd.DataFrame(all_player_data)

# 컬럼명 설정 (1페이지에서 thead 추출)
service = Service(driver_path)
driver = webdriver.Chrome(service=service)
driver.get(url)
time.sleep(2)  # 페이지 로드 대기
soup = BeautifulSoup(driver.page_source, 'html.parser')
player_thead = soup.select('div.record_result > table > thead > tr > th')
player_columns = [th.text.strip() for th in player_thead]
driver.quit()

player.columns = player_columns

# '순위' 컬럼을 인덱스로 설정
player = player.set_index('순위')

# 결과 확인
print(player)

# CSV 파일로 저장
player.to_csv('kbo_picther_data.csv', encoding='utf-8-sig')


     선수명   팀명   ERA  GS Wgs Wgr GF SVO TS GDP   GO   AO GO/AO
순위                                                           
1     네일  KIA  2.53  26  12   0  0   0  0  16  188  115  1.63
2     하트   NC  2.69  26  13   0  0   0  0   7  142  131  1.08
3     반즈   롯데  3.35  25   9   0  0   0  0  18  154  103  1.50
4    후라도   키움  3.36  30  10   0  0   0  0  21  211  166  1.27
5     코너   삼성  3.43  28  11   0  1   0  0   7  123  186  0.66
6    원태인   삼성  3.66  28  15   0  1   0  0   6  160  186  0.86
7   헤이수스   키움  3.68  30  13   0  0   0  0  12  149  169  0.88
8    손주영   LG  3.79  27   9   0  0   0  0  16  173  125  1.38
9    레예스   삼성  3.81  26  11   0  0   0  0   7  144  158  0.91
10   윌커슨   롯데  3.84  32  12   0  1   0  0  11  175  232  0.75
11   류현진   한화  3.87  28  10   0  0   0  0  18  182  140  1.30
12   양현종  KIA  4.10  29  11   0  2   0  0  14  174  201  0.87
13  쿠에바스   KT  4.10  31   7   0  0   0  0   9  155  199  0.78
14    엔스   LG  4.19  30  13   0  0   0  0  12  167  156  1.07
15    곽빈

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time

# WebDriver 설정
driver_path = "C://Users//user//jupyter//kbl 데이터 크롤링//chromedriver-win64//chromedriver-win64//chromedriver.exe"
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# 사이트 접속
url = "https://www.koreabaseball.com/Record/Player/HitterBasic/BasicOld.aspx?sort=HRA_RT"
driver.get(url)

# 페이지 로드 대기
time.sleep(3)

# `select` 요소에서 KIA 팀 선택
team_select_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "cphContents_cphContents_cphContents_ddlTeam_ddlTeam"))
)
team_select = Select(team_select_element)

# KIA 팀 선택 (option value="HT")
team_select.select_by_value("HT")

# 페이지 로드 대기
time.sleep(3)

# HTML 파싱
soup = BeautifulSoup(driver.page_source, "html.parser")

# 테이블 추출
table_div = soup.find("div", class_="record_result")
if table_div:
    table = table_div.find("table")
    tbody = table.find("tbody")

    # 테이블 데이터 추출
    rows = tbody.find_all("tr")
    all_data = []
    for row in rows:
        cells = row.find_all("td")
        row_data = [cell.text.strip() for cell in cells]
        all_data.append(row_data)

    # 데이터프레임 생성
    df = pd.DataFrame(all_data)

    # 컬럼명 설정 (헤더 추출)
    thead = table.find("thead")
    if thead:
        headers = [th.text.strip() for th in thead.find_all("th")]
        df.columns = headers

    # 결과 확인
    print(df.head())

    # CSV 파일로 저장
    df.to_csv("kia_player_data.csv", encoding="utf-8-sig", index=False)
else:
    print("테이블을 찾을 수 없습니다. HTML 구조를 확인하세요.")

# WebDriver 종료
driver.quit()


  순위  선수명   팀명    AVG    G   PA   AB    H  2B  3B  HR  RBI  SB CS  BB HBP  \
0  1  윤도현  KIA  0.407    6   27   27   11   2   0   1    8   1  0   0   0   
1  2  김두현  KIA  0.400    3    6    5    2   0   0   0    0   0  0   0   1   
2  3  김도영  KIA  0.347  141  625  544  189  29  10  38  109  40  4  66   7   
3  4  김선빈  KIA  0.329  116  466  423  139  23   0   9   57   5  1  30   5   
4  5  서건창  KIA  0.310   94  248  203   63  14   1   1   26   3  1  36   3   

    SO GDP   E  
0    9   0   0  
1    0   0   2  
2  110  10  30  
3   39  13  10  
4   31   2   5  


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time

# WebDriver 설정
driver_path = "C://Users//user//jupyter//kbl 데이터 크롤링//chromedriver-win64//chromedriver-win64//chromedriver.exe"
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# 사이트 접속
url = "https://www.koreabaseball.com/Record/Player/HitterBasic/BasicOld.aspx?sort=HRA_RT"
driver.get(url)

# 페이지 로드 대기
time.sleep(3)

# `select` 요소에서 KIA 팀 선택
team_select_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "cphContents_cphContents_cphContents_ddlTeam_ddlTeam"))
)
team_select = Select(team_select_element)

# KIA 팀 선택 (option value="HT")
team_select.select_by_value("HT")

# 페이지 로드 대기
time.sleep(3)

all_data = []

while True:
    # HTML 파싱
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 테이블 추출
    table_div = soup.find("div", class_="record_result")
    if table_div:
        table = table_div.find("table")
        tbody = table.find("tbody")

        # 테이블 데이터 추출
        rows = tbody.find_all("tr")
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            all_data.append(row_data)

        # '다음 페이지' 버튼 찾기
        next_button = soup.find("a", class_="next")
        if next_button and 'disabled' not in next_button.get('class', []):
            # 다음 페이지로 이동
            driver.find_element(By.LINK_TEXT, '다음').click()
            time.sleep(3)  # 페이지 로드 대기
        else:
            break
    else:
        print("테이블을 찾을 수 없습니다. HTML 구조를 확인하세요.")
        break

# 데이터프레임 생성 및 컬럼명 설정
df = pd.DataFrame(all_data)
thead = table.find("thead")
if thead:
    headers = [th.text.strip() for th in thead.find_all("th")]
    df.columns = headers

# 결과 확인
print(df.head())

# CSV 파일로 저장
df.to_csv("kia_player_data.csv", encoding="utf-8-sig", index=False)

# WebDriver 종료
driver.quit()


  순위  선수명   팀명    AVG    G   PA   AB    H  2B  3B  HR  RBI  SB CS  BB HBP  \
0  1  윤도현  KIA  0.407    6   27   27   11   2   0   1    8   1  0   0   0   
1  2  김두현  KIA  0.400    3    6    5    2   0   0   0    0   0  0   0   1   
2  3  김도영  KIA  0.347  141  625  544  189  29  10  38  109  40  4  66   7   
3  4  김선빈  KIA  0.329  116  466  423  139  23   0   9   57   5  1  30   5   
4  5  서건창  KIA  0.310   94  248  203   63  14   1   1   26   3  1  36   3   

    SO GDP   E  
0    9   0   0  
1    0   0   2  
2  110  10  30  
3   39  13  10  
4   31   2   5  


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time

# WebDriver 설정
driver_path = "C://Users//user//jupyter//kbl 데이터 크롤링//chromedriver-win64//chromedriver-win64//chromedriver.exe"
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# 사이트 접속
url = "https://www.koreabaseball.com/Record/Player/HitterBasic/Basic2.aspx"
driver.get(url)

# 페이지 로드 대기
time.sleep(3)

# `select` 요소에서 KIA 팀 선택
team_select_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "cphContents_cphContents_cphContents_ddlTeam_ddlTeam"))
)
team_select = Select(team_select_element)

# KIA 팀 선택 (option value="HT")
team_select.select_by_value("HT")

# 페이지 로드 대기
time.sleep(3)

# HTML 파싱
soup = BeautifulSoup(driver.page_source, "html.parser")

# 테이블 추출
table_div = soup.find("div", class_="record_result")
if table_div:
    table = table_div.find("table")
    tbody = table.find("tbody")
    all_data = []

    # 테이블 데이터 추출
    rows = tbody.find_all("tr")
    for row in rows:
        cells = row.find_all("td")
        row_data = [cell.text.strip() for cell in cells]
        all_data.append(row_data)

    # 데이터프레임 생성 및 컬럼명 설정
    thead = table.find("thead")
    if thead:
        headers = [th.text.strip() for th in thead.find_all("th")]
        df = pd.DataFrame(all_data, columns=headers)
    else:
        df = pd.DataFrame(all_data)

    # 결과 확인
    print(df.head())

    # CSV 파일로 저장
    df.to_csv("kia_player_data.csv", encoding="utf-8-sig", index=False)
else:
    print("테이블을 찾을 수 없습니다. HTML 구조를 확인하세요.")

# WebDriver 종료
driver.quit()



  순위  선수명   팀명    AVG  BB IBB HBP   SO GDP    SLG    OBP    OPS  MH   RISP  \
0  1  윤도현  KIA  0.407   0   0   0    9   0  0.593  0.407  1.000   5  0.500   
1  2  김두현  KIA  0.400   0   0   1    0   0  0.400  0.500  0.900   0  0.000   
2  3  김도영  KIA  0.347  66   7   7  110  10  0.647  0.420  1.067  58  0.317   
3  4  김선빈  KIA  0.329  30   0   5   39  13  0.447  0.380  0.827  42  0.350   
4  5  서건창  KIA  0.310  36   0   3   31   2  0.404  0.416  0.820  11  0.344   

   PH-BA  
0  0.000  
1  0.000  
2  0.500  
3  0.500  
4  0.346  


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time

# WebDriver 설정
driver_path = "C://Users//user//jupyter//kbl 데이터 크롤링//chromedriver-win64//chromedriver-win64//chromedriver.exe"
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# 사이트 접속
url = "https://www.koreabaseball.com/Record/Player/HitterBasic/BasicOld.aspx?sort=HRA_RT"
driver.get(url)

# 페이지 로드 대기
time.sleep(3)

# `select` 요소에서 KIA 팀 선택
team_select_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "cphContents_cphContents_cphContents_ddlTeam_ddlTeam"))
)
team_select = Select(team_select_element)

# KIA 팀 선택 (option value="HT")
team_select.select_by_value("SS")

# 페이지 로드 대기
time.sleep(3)

# HTML 파싱
soup = BeautifulSoup(driver.page_source, "html.parser")

# 테이블 추출
table_div = soup.find("div", class_="record_result")
if table_div:
    table = table_div.find("table")
    tbody = table.find("tbody")

    # 테이블 데이터 추출
    rows = tbody.find_all("tr")
    all_data = []
    for row in rows:
        cells = row.find_all("td")
        row_data = [cell.text.strip() for cell in cells]
        all_data.append(row_data)

    # 데이터프레임 생성
    df = pd.DataFrame(all_data)

    # 컬럼명 설정 (헤더 추출)
    thead = table.find("thead")
    if thead:
        headers = [th.text.strip() for th in thead.find_all("th")]
        df.columns = headers

    # 결과 확인
    print(df.head())

    # CSV 파일로 저장
    df.to_csv("samsung_player_data.csv", encoding="utf-8-sig", index=False)
else:
    print("테이블을 찾을 수 없습니다. HTML 구조를 확인하세요.")

# WebDriver 종료
driver.quit()


  순위   선수명  팀명    AVG    G   PA   AB    H  2B 3B  HR  RBI  SB CS  BB HBP  SO  \
0  1   구자욱  삼성  0.343  129  568  493  169  39  1  33  115  13  4  55  12  73   
1  2  카데나스  삼성  0.333    7   25   24    8   2  0   2    5   0  1   1   0   6   
2  3   김지찬  삼성  0.316  135  535  453  143  16  3   3   36  42  4  60   9  40   
3  4   강민호  삼성  0.303  136  452  403  122  19  1  19   77   3  0  35   8  52   
4  5   김헌곤  삼성  0.302  117  311  281   85   8  1   9   34   4  2  22   3  50   

  GDP  E  
0   6  1  
1   1  0  
2   5  3  
3  17  2  
4   5  1  
